In [ ]:
import os

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit

In [ ]:
class FLI_Prior:
    def __init__(self):
        self.tau1_mean_hyperprior_mean = np.log(0.2)
        self.tau1_mean_hyperprior_std = 0.7
        self.tau1_std_hyperprior_mean = -1
        self.tau1_std_hyperprior_std = 0.1

        self.delta_tau_mean_hyperprior_mean = np.log(1.2)
        self.delta_tau_mean_hyperprior_std = 0.5
        self.delta_tau_std_hyperprior_mean = -2
        self.delta_tau_std_hyperprior_std = 0.1

        self.a_mean_hyperprior_mean = 0
        self.a_mean_hyperprior_std = 1
        self.a_std_hyperprior_mean = -1
        self.a_std_hyperprior_std = 1

        self.sample_global()

    def sample_global(self):
        self.log_tau_G = np.random.normal(self.tau1_mean_hyperprior_mean, self.tau1_mean_hyperprior_std)
        self.log_sigma_tau_G = np.random.normal(self.tau1_std_hyperprior_mean, self.tau1_std_hyperprior_std)

        self.log_delta_tau_G = np.random.normal(self.delta_tau_mean_hyperprior_mean, self.delta_tau_mean_hyperprior_std)
        self.log_delta_sigma_tau_G = np.random.normal(self.delta_tau_std_hyperprior_mean, self.delta_tau_std_hyperprior_std)

        self.a_mean = np.random.normal(self.a_mean_hyperprior_mean, self.a_mean_hyperprior_std)
        self.a_log_std = np.random.normal(self.a_std_hyperprior_mean, self.a_std_hyperprior_std)
        self.A_mean = expit(self.a_mean)

        return dict(log_tau_G=self.log_tau_G, log_sigma_tau_G=self.log_sigma_tau_G,
                    log_delta_tau_G=self.log_delta_tau_G, log_delta_sigma_tau_G=self.log_delta_sigma_tau_G,
                    a_mean=self.a_mean, a_log_std=self.a_log_std, A_mean=self.A_mean)

    def get_local_variances(self):
        var_tau_L = np.exp(2 * self.log_tau_G + np.exp(self.log_sigma_tau_G)**2) * (np.exp(np.exp(self.log_sigma_tau_G)**2) - 1)
        var_delta = np.exp(2 * self.log_delta_tau_G + np.exp(self.log_delta_sigma_tau_G)**2) * (np.exp(np.exp(self.log_delta_sigma_tau_G)**2) - 1)
        tau_L1_std = np.sqrt(var_tau_L)
        tau_L2_std = np.sqrt(var_tau_L + var_delta)
        return dict(tau_L1_std=tau_L1_std, tau_L2_std=tau_L2_std)


    def sample_local(self, n_local_samples):
        log_tau_L = np.random.normal(self.log_tau_G, np.exp(self.log_sigma_tau_G), size=n_local_samples)
        log_delta_tau_L = np.random.normal(self.log_delta_tau_G, np.exp(self.log_delta_sigma_tau_G), size=n_local_samples)
        a_l = np.random.normal(self.a_mean, np.exp(self.a_log_std), size=n_local_samples)

        tau_L = np.exp(log_tau_L)
        tau_L_2 = tau_L + np.exp(log_delta_tau_L)
        A_L = expit(a_l)
        return dict(tau_L=tau_L, tau_L_2=tau_L_2, A_L=A_L,
                    log_tau_L=log_tau_L, log_delta_tau_L=log_delta_tau_L, a_l=a_l)

In [ ]:
# distribution
n_global_samples = 1000

prior = FLI_Prior()

log_tau_G = []
log_sigma_tau_G = []
log_delta_tau_G = []
log_delta_sigma_tau_G = []
a_mean = []
a_log_std = []
A_mean = []
tau_L1_std = []
tau_L2_std = []

for i in range(n_global_samples):
    sample = prior.sample_global()
    log_tau_G.append(sample['log_tau_G'])
    log_sigma_tau_G.append(sample['log_sigma_tau_G'])
    log_delta_tau_G.append(sample['log_delta_tau_G'])
    log_delta_sigma_tau_G.append(sample['log_delta_sigma_tau_G'])
    a_mean.append(sample['a_mean'])
    a_log_std.append(sample['a_log_std'])
    A_mean.append(sample['A_mean'])
    tau_std = prior.get_local_variances()
    tau_L1_std.append(tau_std['tau_L1_std'])
    tau_L2_std.append(tau_std['tau_L2_std'])
log_tau_G = np.array(log_tau_G)
log_sigma_tau_G = np.array(log_sigma_tau_G)
log_delta_tau_G = np.array(log_delta_tau_G)
log_delta_sigma_tau_G = np.array(log_delta_sigma_tau_G)
a_mean = np.array(a_mean)
a_log_std = np.array(a_log_std)
A_mean = np.array(A_mean)
tau_L1_std = np.array(tau_L1_std)
tau_L2_std = np.array(tau_L2_std)


tau_G = np.exp(log_tau_G)
tau_G2 = tau_G + np.exp(log_delta_tau_G)

plt.title(r'Global Parameters $\tau$')
plt.hist(tau_G, bins=30, alpha=0.7, density=True, label=r'$\tau^G$')
plt.hist(tau_G2, bins=30, alpha=0.7, density=True, label=r'$\tau^G_2$')
plt.legend()
plt.show()

plt.title(r'Global Parameters $\sigma_{\tau}$')
plt.hist(tau_L1_std, bins=30, alpha=0.7, density=True, label=r'$\sigma_{\tau^G}$')
plt.hist(tau_L2_std, bins=30, alpha=0.7, density=True, label=r'$\sigma_{\tau^G_2}$')
plt.legend()
plt.show()

plt.title(r'Global Parameters unbounded $a$')
plt.hist(a_mean, bins=30, alpha=0.7, density=True, label=r'$a$ mean')
plt.hist(np.exp(a_log_std), bins=30, alpha=0.7, density=True, label=r'$a$ std')
plt.legend()
plt.show()

plt.title(r'Global Parameters $A$')
plt.hist(A_mean, bins=30, alpha=0.7, density=True, label=r'$A$')
plt.legend()
plt.show()

In [ ]:
global_sample = prior.sample_global()
local_samples = prior.sample_local(1000)
tau_std = prior.get_local_variances()
tau_L1_std = tau_std['tau_L1_std']
tau_L2_std = tau_std['tau_L2_std']

tau_L = local_samples['tau_L']
tau_L_2 = local_samples['tau_L_2']
log_delta_tau_L = local_samples['log_delta_tau_L']
A_L = local_samples['A_L']
a_l = local_samples['a_l']

tau_G = np.exp(global_sample['log_tau_G'])
tau_G2 = tau_G + np.exp(global_sample['log_delta_tau_G'])

plt.title(r'Local Parameters $\tau$')
plt.axvline(tau_G, color='red', linestyle='-', label=r'$\tau^G$')
plt.axvline(tau_G-tau_L1_std, color='red', linestyle='--')
plt.axvline(tau_G+tau_L1_std, color='red', linestyle='--')
plt.axvline(tau_G2, color='green', linestyle='-', label=r'$\tau^G_2$')
plt.axvline(tau_G2-tau_L2_std, color='green', linestyle='--')
plt.axvline(tau_G2+tau_L2_std, color='green', linestyle='--')
plt.hist(tau_L, bins=30, density=True, label=r'$\tau^L$')
plt.hist(tau_L_2, bins=30, density=True, label=r'$\tau^L_2$')
plt.legend()
plt.show()

plt.title(r'Local Parameters $A$')
plt.hist(A_L, bins=30, density=True, label=r'$A^L$')
plt.legend()
plt.xlim(0,1)
plt.show()

plt.title(r'Local Parameters $a$')
plt.hist(a_l, bins=30, density=True, label=r'$a^L$')
plt.legend()
plt.show()

In [ ]:
def decay_gen(img, irf, noise, gw, n_local_samples):  # IRF, Gate_width (in ps)
    a, b = np.shape(img)
    a1, b1, c1 = np.shape(irf)
    a2, b2, c2 = np.shape(noise)
    t = np.linspace(0, c1* (gw  * (10 ** -3)), c1)
    t_minus = np.multiply(t, -1)
    #tau_L, tau_L_2, A_L = pram1.generate_local_parameters(n_local_samples)
    prior_samples = prior.sample_local(n_local_samples)
    tau_L, tau_L_2, A_L = prior_samples['tau_L'], prior_samples['tau_L_2'], prior_samples['A_L']
    tau1 = np.reshape(tau_L, (a, b))
    tau2 = np.reshape(tau_L_2, (a, b))
    frac1 = np.reshape(A_L, (a, b))
    frac2 = 1 - frac1
    dec = np.zeros([a1, b1, c1])
    A = np.zeros([a1, b1, c1])
    B = np.zeros([a1, b1, c1])
    irf_out = np.zeros([a, b, c1])
    dec_conv =  np.zeros([a, b, c1])
    for i in range(a):
        for j in range(b):
            if tau1[i, j] != 0:               
                A[i, j, :] = np.multiply(frac1[i, j], np.exp(np.divide(t_minus, tau1[i, j])))
            if tau2[i, j] != 0:   
                B [i, j, :] = np.multiply(frac2[i, j], np.exp(np.divide(t_minus, tau2[i, j])))
            dec[i, j, :] = A[i, j, :] + B [i, j, :]
            irf_out[i, j, :] = norm1D(irf[np.random.randint(a1), np.random.randint(b1), :])
            dec_conv[i, j, :] = conv_dec(norm1D(np.squeeze(dec[i, j, :])), np.squeeze(irf_out[i, j, :]))
            dec_conv[i, j, :] = norm1D(np.squeeze(dec_conv[i, j, :]))*img[i,j]           
            dec_conv[i, j, :] += noise[i,j,:] 
    return dec_conv, prior_samples

def tiff2tpsfs(p):
    k = os.listdir(p)
    le = len(k)
    img0 = mpimg.imread(os.path.join(p, k[0]))
    (a, b) = np.shape(img0)
    tpsfs = np.zeros([a, b, le - 1])
    for count, file in enumerate(os.listdir(p)):
        # Check whether file is in text format or not
        if file.endswith(".tif"):
            imt = mpimg.imread(os.path.join(p, file))
            tpsfs[:, :, count] = imt
    inten = np.sum(tpsfs, axis=2)
    return tpsfs, inten

def norm1D(fn):
    if np.amax(fn) == 0:
        nfn = fn
    else:
        nfn = np.divide(fn, np.amax(fn))
    return nfn

def conv_dec(dec, irf):
    c = np.shape(dec)
#     conv = np.zeros((2 * c - 1,1))
    conv = np.convolve(dec, irf)
    conv = conv[:len(dec)]
    return conv


In [ ]:
ns = np.load('system_noise.npy')
pIRF = np.load('IRF.npy')

In [ ]:
ns.shape

In [ ]:
print(pIRF.shape)
plt.imshow(np.sum(pIRF, axis=2))
plt.show()
a = np.random.randint(ns.shape[0])
b = np.random.randint(ns.shape[1])
print(a)
print(b)
plt.plot(ns[a,b,:])
plt.show()

### For Data generation

In [ ]:
gw = 40
data_size = 5
img_size = (520, 696)

# global parameters
F_log_tau_G = np.zeros((data_size, 1))
F_log_sigma_tau_G = np.zeros((data_size, 1))
F_log_Delta_tau_G = np.zeros((data_size, 1))
F_log_Delta_sigma_tau_G = np.zeros((data_size, 1))
F_a_mean = np.zeros((data_size, 1))
F_a_log_std = np.zeros((data_size, 1))

# data and local parameters
F_dec_conv = np.zeros((data_size,img_size[0], img_size[1], pIRF.shape[2]))
F_tau1 = np.zeros((data_size, img_size[0]*img_size[1]))
F_tau2 = np.zeros((data_size, img_size[0]*img_size[1]))
F_frac1 = np.zeros((data_size, img_size[0]*img_size[1]))

for i in range(data_size):
    global_sample = prior.sample_global()
    F_log_tau_G[i], F_log_sigma_tau_G[i] = global_sample['log_tau_G'], global_sample['log_sigma_tau_G']
    F_log_Delta_tau_G[i], F_log_Delta_sigma_tau_G[i] = global_sample['log_delta_tau_G'], global_sample['log_delta_sigma_tau_G']
    F_a_mean[i], F_a_log_std[i] = global_sample['a_mean'], global_sample['a_log_std']

    img = np.random.randint(0, high=1000, size=(img_size[0], img_size[1]))
    F_dec_conv[i], prior_samples = decay_gen(img, pIRF, ns, gw, n_local_samples=img_size[0]*img_size[1])
    F_tau1[i], F_tau2[i], F_frac1[i] = prior_samples['log_tau_L'], prior_samples['log_delta_tau_L'], prior_samples['a_l']

In [ ]:
data = F_dec_conv[np.random.randint(data_size),:,:,:]
x = np.random.randint(data.shape[0])
y = np.random.randint(data.shape[1])
print(x)
print(y)
plt.plot(data[x,y,:])
plt.show()

In [ ]:
plt.figure()
for i in range(10):
    for j in range(10):
        slice_data = data[i+90, j+90, :]  # Extract the data along axis 3 for (i, j)

        plt.plot(range(data.shape[2]), slice_data) # plot the 201 values
#         plt.title(f"Data at (i={i}, j={j}) along axis 3")
#         plt.xlabel("Axis 3 Index")
#         plt.ylabel("Data Value")
        plt.grid(True)
plt.show()